In [5]:
import numpy as np
import pandas as pd
import pickle
import tflscripts
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

configuration = tflscripts.read_configuration()

In [37]:
df_results = pd.read_pickle('results_with_accuracy_classified.p')
df_results = df_results.loc[df_results.accuracy_negative >= 0.95]
df_results = df_results.loc[(df_results.accuracy_bin == 1) | (df_results.predictions == 1)]

In [33]:
def key_for_test(source_dataset,
                    source_device,
                    target_dataset,
                    target_device,
                    label,
                    features,
                    classifier):
    return source_dataset + source_device + target_dataset + target_device + \
        label + features + classifier

def key_for_result_row(row):
    return key_for_test(source_dataset=row['source_dataset'],
                       source_device=row['source_device'],
                       target_dataset=row['target_dataset'],
                       target_device=row['target_device'],
                       label=row['label'],
                       features=row['features'],
                       classifier=row['classifier'])

def key_for_test_result(result):
    return key_for_test(source_dataset=result.source_dataset,
                       source_device=result.source_device,
                       target_dataset=result.target_dataset,
                       target_device=result.target_device,
                       label=result.label_name(),
                       features=result.features,
                       classifier=result.classifier)

def test_results_for_df(df_results):
    wanted_test_results = {}
    for i, row in df_results.iterrows():
        key = key_for_result_row(row)
        wanted_test_results[key] = i

    found_test_results = {}
    for dataset in configuration['device_roles']:
        for device in configuration['device_roles'][dataset]:
            test_set = tflscripts.TestSet(name='_'.join([dataset, device]))
            if test_set.exists():
                for result in test_set.get_results():
                    key = key_for_test_result(result)
                    if key in wanted_test_results:
                        result_row_id = wanted_test_results[key]
                        found_test_results[result_row_id] = result

    return found_test_results

def filter_results_by_accuracy(case_results, accuracy_column):
    label_models = {}
    for i, row_result in case_results.iterrows():
        label = row_result['label']
        if label in label_models:
            other_row_result = case_results.loc[case_results.index == label_models[label]].iloc[0]
            if other_row_result[accuracy_column] < row_result[accuracy_column]:
                label_models[label] = i
        else:
            label_models[label] = i

    result_ids = list(label_models.values())
    return result_ids

def majority_vote(row):
    value_counts = row.value_counts()
    value_counts = value_counts[value_counts.index != -1]
    if len(value_counts) == 0:
        return -1
    return value_counts.idxmax()

In [31]:
cases = df_results[['target_device', 'target_dataset']].drop_duplicates()
cases.columns = ['device', 'dataset']
all_test_results = test_results_for_df(df_results)

In [36]:
for _, case in cases.iterrows():
    # find tests for the case
    case_results = df_results.loc[df_results.target_device == case['device']]
    case_results = case_results.loc[case_results.target_dataset == case['dataset']]

    results_ids = filter_results_by_accuracy(case_results, 'accuracy_positive')
    test_results = [all_test_results[i] for i in results_ids]
    df = pd.DataFrame([r.predicted for r in test_results]).T
    df.columns = results_ids

    actual = all_test_results[results_ids[0]].actual_with_all_labels
    majority_votes = [majority_vote(row) for i, row in df.iterrows()]

    df['actual'] = actual
    df['majority_votes'] = majority_votes

    can_predict_labels = [r.label for r in test_results]
    df_known_labels = df.loc[df.actual.isin(can_predict_labels)]
    df_all_predicted = df.loc[df.majority_votes != -1]

    accuracy_known_labels = accuracy_score(df_known_labels['actual'], df_known_labels['majority_votes'])
    accuracy_all_predicted = accuracy_score(df_all_predicted['actual'], df_all_predicted['majority_votes'])
    print(case['dataset'], case['device'], accuracy_known_labels, accuracy_all_predicted)

    for result in test_results:
        print(result.source_dataset, result.source_device, result.label, result.features, result.classifier)

    print()

synergy-final-iter5 128.237.250.218 0.892096545196 0.876336587634
synergy-final-iter1 128.237.248.186 11 ACCEL_|accel_|mag_ SVM
synergy-final-iter4 128.237.250.218 0 MICROPHONE_|microphone LogisticRegression
synergy-final-iter5 128.237.227.76 23 MICROPHONE_|microphone SVM
synergy-final-iter4 128.237.247.190 25 MICROPHONE_|microphone SVM
synergy-final-iter5 128.237.227.76 22 MICROPHONE|microphone|ACCEL_|accel_ LogisticRegression
synergy-final-iter4 128.237.247.190 13 .* LogisticRegression
synergy-final-iter4 128.237.247.190 7 MICROPHONE_|microphone LogisticRegression
synergy-final-iter5 128.237.227.76 21 MICROPHONE_|microphone SVM
synergy-final-iter5 128.237.227.76 1 MICROPHONE_|microphone LogisticRegression
synergy-final-iter4 128.237.250.218 12 MICROPHONE|microphone|ACCEL_|accel_ LogisticRegression

synergy-final-iter1 128.237.254.195 0.959349593496 0.846484935438
synergy-final-iter2 128.237.248.186 0 .* LogisticRegression
synergy-final-iter2 128.237.248.186 1 MICROPHONE_|microphone S

IndexError: list index out of range

In [170]:
can_predict_labels = [r.label for r in results_by_accuracy]
df_known_labels = df.loc[df.actual.isin(can_predict_labels)]
confusion_matrix(df_known_labels.actual, df_known_labels.majority_votes)

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [293,  49,   2,   0,   3,   0,   1,   0,   0,   0],
       [222,   4, 283,   3,   2,   0,   1,   0,   0,   0],
       [ 17,   0,   0,   5,   2,   0,   1,   0,   0,   0],
       [ 27,   0,   0,   0,   1,   0,   0,   0,   0,   0],
       [  2,   0,   2,   7,   2,  16,   0,   0,   0,   0],
       [ 61,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,  13,   2,   0,   0, 156,  32,   0],
       [  2,   0,   7,   1,   1,   0,   0,   0, 287,   0],
       [  0,   0,   1,   0,   0,   0,   0,   0,   0, 542]])

In [22]:
1

1